In [1]:
def main(): 
    import json
    import  graph_construction as function
    import pandas as pd
    import os

    #CARICAMENTO FILE CONFIGURAZIONE
    with open("config.json") as f:
        data=json.load(f)

    #Definizione pathway
    path_data_mutational=data["Paths"]["data_mutational"]
    path_clinical_sample=data["Paths"]["data_clinical_sample"]
    path_clinical_patient=data["Paths"]["data_clinical_patient"]
    column_mutation_name=data["Mutation"]["column_mutation_name"]
    gene_interest=data["Mutation"]["gene_of_interest"]
    path_save=data["Paths"]["output_folder"]
    column_gene=data["Mutation"]["column_gene_name"]
    column_hgvsp=data["Mutation"]["column_hgvsp_short"]
    column_hgvsc=data["Mutation"]["column_hgvsc"]
    column_variant_classification=data["Mutation"]["column_variant_classification"]

    #caricamento dei dataframes
    data_mutational,data_clinical_sample,data_clinical_patient=function.read_file(path_data_mutational,path_clinical_sample,path_clinical_patient)
    
    #creazione delle mappe pazienti e varianti + creazione del grafo
    if column_mutation_name=="":
        data_mutational=function.adding_category_mutation(data_mutational,column_gene,column_hgvsp,column_hgvsc,column_variant_classification)

    map_patients,map_variants,map_consequence=function.create_maps(data_mutational,data,column_mutation_name)
    graph=function.graph_creation(map_patients,map_variants)

    #plot tipi di mutazione geni di interesse
    if len(gene_interest)!=0:
        function.plot_mutation_gene(map_consequence,gene_interest,path_save)

    #clusterizzazione
    seed=function.selected_seed(graph)
    dendro=function.leiden_clustering(graph,seed)

    #gestione parte grafica del grafo (aggiunta colori + file per cytoscape)
    graph=function.adding_graph_color(graph,dendro)
    if not os.path.exists(path_save):
        os.makedirs(path_save)
    function.write_graph_to_cytoscape(graph,path_save)


    #creazione della mappa cluster e attribuzione del cluster i pazienti e alle varianti
    map_cluster=function.map_cluster_creation(graph,dendro)
    map_patients,map_variants=function.adding_cluster_to_map(map_cluster,map_patients,map_variants)
    graph=function.cluster_noded_attributes(graph,map_patients,map_variants)

    #aggiunta delle informazioni cliniche alla mappa dei pazienti
    if len(data_clinical_sample)!=0:
        map_patients=function.enriched_sample_data(data_clinical_sample,map_cluster,map_patients)
    if len(data_clinical_patient)!=0:
        map_patients=function.enriched_patient_data(data_clinical_patient,map_patients)

    #creazione file in cui riassumere le informazioni nei diversi cluster
    function.summary_info(path_save,map_cluster,map_patients)
    function.numerosity_info(path_save,map_cluster)

    #creazione di una mappa con il numero di mutazioni per ogni gene + creazione di due mappe con i valori assoluti e percentuali di distribuzione delle mutazioni, per ciascun gene,
    #nei diversi cluster
    gene_total_count=function.count_gene(graph)
    map_cluster_gene_abs,map_cluster_gene_percent=function.count_gene_abs_percent(map_cluster,gene_total_count,path_save)
    #salvataggio delle percentuali di distribuzione delle mutazioni dei diversi geni nei cluster
    function.genes_single_cluster(map_cluster,path_save)

    function.couple_centroid_element(dendro,map_cluster,path_save)
    function.centroids_cluster(dendro,path_save)


    #**************SEZIONE GRAFICA CONNESSIONI***************
    #defizione del numero di connessioni per ogni variante all'interno del cluster 
    variant_patient_connection_count=function.variant_conncection_patient(dendro)
    #defizione del numero di connessioni per ogni paziente all'interno del cluster
    patient_variant_connection_count=function.patient_connection_variant(dendro)
    graph=function.add_size_node(graph,variant_patient_connection_count)
    # CREAZIONE DI UN FILE "CONNECTION_VARIANT" IN CUI SONO INDICATE IL NUMERO DI VARIANTI COMUNI TRA I VARI PAZIENTI DI UN CLUSTER
    function.file_connection_variant(map_cluster,map_patients,path_save)
    # CREAZIONE DI UN FILE "CONCCECTION_PATIENT" IN CUI SONO INDICATI IL NUMERO DI PAZIENTI COMUNI TRA LE VARIE VARIANTI DI UN CLUSTER
    function.file_connection_patient(map_cluster,map_variants,path_save)
    #plott delle connessioni lato varianti:
    data_connection=pd.read_csv("./tcga_filtred/output/connection_patient.csv",sep="\t")
    #for i in range(len(map_cluster.keys())):
      # function.plot_distance_comutated_cluster_variants(dendro,i,data_connection,path_save)


if __name__=="__main__":
    main()

numero di clusters: 55 Modularità: 0.9587875642357918
Mut_ARID1A_p.S1516Lfs*11
Mut_FAF1_p.M394T
Mut_CELF3_p.R159W
Mut_SLC45A3_p.D116Y
Mut_HPS3_p.P992S
Mut_SI_p.D859G
Mut_UBE2QL1_p.R120W
Mut_DOCK2_p.K1058T
Mut_TREML2_p.P115L
Mut_USP49_p.Y598X
Mut_HTR1E_p.R204Q
Mut_KIAA1217_p.S890Y
Mut_SNX29_p.E589K
Mut_TMEM102_p.R311C
Mut_RBBP8NL_p.R205Q
Mut_TBC1D25_p.L544S
Mut_RLIM_p.R510X
Mut_GABRQ_p.P191S
Mut_TAFA4_p.H59H
Mut_FBRSL1_p.S830S
Mut_GNB5_p.F364F
C3N-01388
Mut_KRAS_p.G12D
Mut_CLSPN_p.T98I
Mut_HIVEP3_p.R1747Q
Mut_AXDND1_p.N651K
Mut_INTS7_p.N814K
Mut_DYSF_p.R468H
Mut_NEK10_p.T1099K
Mut_KIAA1257_p.S283G
Mut_DMP1_p.Q510K
Mut_SYNPO2_p.R37X
Mut_SLC12A7_p.T1080N
Mut_CLVS2_p.R70C
Mut_VWC2_p.G86D
Mut_RAB19_p.N27S
Mut_KCNH2_p.E289K
Mut_CNTLN_p.R604X
Mut_WDFY4_p.Q1966Pfs*50
Mut_PALD1_p.R485W
Mut_AMBRA1_p.R182W
Mut_OR4X1_p.G39V
Mut_RTN3_p.S105Kfs*50
Mut_KMT2D_p.R487Q
Mut_KRT79_p.E275G
Mut_STX2_p.V236I
Mut_NRXN3_p.A185T
Mut_PRC1_p.A168Gfs*3
Mut_ZNF668_p.R187Q
Mut_SLC5A2_p.G418D
Mut_NDRG4_p.R215W
Mut_CB

CARICAMENTO DATI

In [2]:
import json
import  graph_construction as function
import os

#CARICAMENTO FILE CONFIGURAZIONE
with open("config.json") as f:
    data=json.load(f)

#Definizione pathway
path_data_mutational=data["Paths"]["data_mutational"]
path_clinical_sample=data["Paths"]["data_clinical_sample"]
path_clinical_patient=data["Paths"]["data_clinical_patient"]
column_mutation_name=data["Mutation"]["column_mutation_name"]
gene_interest=data["Mutation"]["gene_of_interest"]
path_save=data["Paths"]["output_folder"]
column_gene=data["Mutation"]["column_gene_name"]
column_hgvsp=data["Mutation"]["column_hgvsp_short"]
column_hgvsc=data["Mutation"]["column_hgvsc"]
column_variant_classification=data["Mutation"]["column_variant_classification"]
sample_name=data["Clinical_data"]["column_sample_name"]
patient_name=data["Clinical_data"]["column_patient_name"]

In [3]:
#caricamento dei dataframes
data_mutational,data_clinical_sample,data_clinical_patient=function.read_file(path_data_mutational,path_clinical_sample,path_clinical_patient)

DIZIONARIO PAZIENTI-DIZIONARIO VARIANTI-COSTRUZIONE GRAFO

In [4]:
#creazione delle mappe pazienti e varianti + creazione del grafo
if column_mutation_name=="":
    data_mutational=function.adding_category_mutation(data_mutational,column_gene,column_hgvsp,column_hgvsc,column_variant_classification,)

map_patients,map_variants,map_consequence=function.create_maps(data_mutational,data,column_mutation_name)

graph=function.graph_creation(map_patients,map_variants)


In [5]:
#plot tipi di mutazione geni di interesse
if len(gene_interest)!=0:
    function.plot_mutation_gene(map_consequence,gene_interest)

CLUSTERIZZAZIONE

In [6]:
#clusterizzazione
seed=function.selected_seed(graph)
dendro=function.leiden_clustering(graph,seed)


numero di clusters: 55 Modularità: 0.9587875642357918


In [7]:
#gestione parte grafica del grafo (aggiunta colori + file per cytoscape)
graph=function.adding_graph_color(graph,dendro)
if not os.path.exists(path_save):
    os.makedirs(path_save)
function.write_graph_to_cytoscape(graph,path_save)



DIZIONARIO CLUSTER

In [8]:
#creazione della mappa cluster e attribuzione del cluster i pazienti e alle varianti
map_cluster=function.map_cluster_creation(graph,dendro)

map_patients,map_variants=function.adding_cluster_to_map(map_cluster,map_patients,map_variants)

graph=function.cluster_noded_attributes(graph,map_patients,map_variants)

INFORMAZIONI CLINICHE

In [9]:
#aggiunta delle informazioni cliniche alla mappa dei pazienti
if len(data_clinical_sample)!=0:
    map_patients=function.enriched_sample_data(data_clinical_sample,map_cluster,map_patients,sample_name,patient_name)
if len(data_clinical_patient)!=0:
    map_patients=function.enriched_patient_data(data_clinical_patient,map_patients,patient_name)

FILE DI SINTESI

In [10]:
#creazione file in cui riassumere le informazioni nei diversi cluster
function.summary_info(path_save,map_cluster,map_patients,patient_name)
function.numerosity_info(path_save,map_cluster)

GENI

In [11]:
#creazione di una mappa con il numero di mutazioni per ogni gene + creazione di due mappe con i valori assoluti e percentuali di distribuzione delle mutazioni, per ciascun gene,
#nei diversi cluster
gene_total_count=function.count_gene(graph)
map_cluster_gene_abs,map_cluster_gene_percent=function.count_gene_abs_percent(map_cluster,gene_total_count,path_save)

#salvataggio delle percentuali di distribuzione delle mutazioni dei diversi geni nei cluster
function.genes_single_cluster(map_cluster,path_save)

In [12]:
function.creation_cluster_clinical_data(map_patients,path_save)

,Age,BMI,CDKN2A_mutation,Histologic_Grade,KRAS_mutation,Path_Stage_pN,Path_Stage_pT,SMAD4_mutation,Sex,Stage,TP53_mutation,Tobacco_smoking_history,Tumor_Size_cm,Tumor_necrosis,case_id,cluster
0,53,18.00,0,G2 Moderately differentiated,1,pN2,pT2,0,Female,Stage III,0,non-smoker,3.5,Present,C3N-01388,0
1,80,31.00,1,G2 Moderately differentiated,1,pN1,pT3,1,Female,Stage III,0,past smoker,4.0,Not identified,C3N-01383,2
2,56,17.00,0,G2 Moderately differentiated,1,pN1,pT1,0,Female,Stage II,1,non-smoker,1.7,Not identified,C3N-03439,4
3,64,18.00,1,G2 Moderately differentiated,1,pN1,pT2,0,Female,Stage III,1,non-smoker,3.3,Not identified,C3N-03006,3
4,63,23.00,0,G2 Moderately differentiated,1,pN1,pT3,0,Female,Stage III,1,non-smoker,2.5,Not identified,C3N-01165,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,71,26.85,1,G3 Poorly differentiated,1,pN1,pT3,0,Male,Stage II,1,non-smoker,3.4,Present,C3L-01328,1
57,52,26.84,0,G3 Poorly differentiated,1,pN1,pT3,0,Female,Stage II,1,non-smoker,6.0,Not identified,C3L-02613,52
58,71,27.43,1,G2 Moderately differentiated,0,pN0,pT2,0,Male,Stage I,1,current smoker,2.5,Not identified,C3N-01012,53
59,55,31.92,1,G2 Moderately differentiated,0,pN1,pT3,0,Male,Stage II,1,non-smoker,3.5,Not identified,C3N-00198,14


In [19]:
 #function.couple_centroid_element(dendro,map_cluster,path_save)
function.centroids_cluster(dendro,path_save)

Mut_ARID1A_p.S1516Lfs*11
Mut_FAF1_p.M394T
Mut_CELF3_p.R159W
Mut_SLC45A3_p.D116Y
Mut_HPS3_p.P992S
Mut_SI_p.D859G
Mut_UBE2QL1_p.R120W
Mut_DOCK2_p.K1058T
Mut_TREML2_p.P115L
Mut_USP49_p.Y598X
Mut_HTR1E_p.R204Q
Mut_KIAA1217_p.S890Y
Mut_SNX29_p.E589K
Mut_TMEM102_p.R311C
Mut_RBBP8NL_p.R205Q
Mut_TBC1D25_p.L544S
Mut_RLIM_p.R510X
Mut_GABRQ_p.P191S
Mut_TAFA4_p.H59H
Mut_FBRSL1_p.S830S
Mut_GNB5_p.F364F
C3N-01388
Mut_KRAS_p.G12D
Mut_CLSPN_p.T98I
Mut_HIVEP3_p.R1747Q
Mut_AXDND1_p.N651K
Mut_INTS7_p.N814K
Mut_DYSF_p.R468H
Mut_NEK10_p.T1099K
Mut_KIAA1257_p.S283G
Mut_DMP1_p.Q510K
Mut_SYNPO2_p.R37X
Mut_SLC12A7_p.T1080N
Mut_CLVS2_p.R70C
Mut_VWC2_p.G86D
Mut_RAB19_p.N27S
Mut_KCNH2_p.E289K
Mut_CNTLN_p.R604X
Mut_WDFY4_p.Q1966Pfs*50
Mut_PALD1_p.R485W
Mut_AMBRA1_p.R182W
Mut_OR4X1_p.G39V
Mut_RTN3_p.S105Kfs*50
Mut_KMT2D_p.R487Q
Mut_KRT79_p.E275G
Mut_STX2_p.V236I
Mut_NRXN3_p.A185T
Mut_PRC1_p.A168Gfs*3
Mut_ZNF668_p.R187Q
Mut_SLC5A2_p.G418D
Mut_NDRG4_p.R215W
Mut_CBFA2T3_p.R638H
Mut_ZNF276_p.A308S
Mut_TP53_p.R248Q
Mut_

In [16]:
#defizione del numero di connessioni per ogni variante all'interno del cluster 
variant_patient_connection_count=function.variant_conncection_patient(dendro)
#defizione del numero di connessioni per ogni paziente all'interno del cluster
patient_variant_connection_count=function.patient_connection_variant(dendro)

In [17]:
graph=function.add_size_node(graph,variant_patient_connection_count)

In [18]:
# CREAZIONE DI UN FILE "CONNECTION_VARIANT" IN CUI SONO INDICATE IL NUMERO DI VARIANTI COMUNI TRA I VARI PAZIENTI DI UN CLUSTER
function.file_connection_variant(map_cluster,map_patients,path_save)

# CREAZIONE DI UN FILE "CONCCECTION_PATIENT" IN CUI SONO INDICATI IL NUMERO DI PAZIENTI COMUNI TRA LE VARIE VARIANTI DI UN CLUSTER
function.file_connection_patient(map_cluster,map_variants,path_save)

In [20]:
import pandas as pd
data_connection=pd.read_csv("./pdac/output/connection_patient.csv",sep="\t")

for i in range(len(map_cluster.keys())):
    function.plot_distance_comutated_cluster_variants(dendro,i,data_connection,path_save)